In [3]:
# Here we are using the same Skeleton of chatbot and improving it by adding loops and conditions to store the chat history.
from langgraph.graph import StateGraph,START,END
from typing import TypedDict, Literal
from langgraph.checkpoint.memory import InMemorySaver
import time

In [4]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str
    step3: str


In [5]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}


def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(30)  # Simulate long-running hang
    return {"step2": "done"}


def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"step3": 'done'}

In [6]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.add_edge(START, "step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)


In [7]:
try:
    print("▶ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"},config={"configurable": {"thread_id": "thread-1"}}
    )
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")


▶ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
❌ Kernel manually interrupted (crash simulated).


In [8]:
graph.get_state({"configurable": {"thread_id": "thread-1"}})

StateSnapshot(values={'input': 'start', 'step1': 'done'}, next=('step_2',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e16a5-db5c-67fe-8001-1b604ab24ab1'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-12-25T08:19:00.346265+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e16a5-db5a-6104-8000-80438cf20afc'}}, tasks=(PregelTask(id='4f1f6d11-7a59-fe0e-d080-1e36c3d70ca5', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=())

In [9]:
list(graph.get_state_history({"configurable": {"thread_id": "thread-1"}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done'}, next=('step_2',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e16a5-db5c-67fe-8001-1b604ab24ab1'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-12-25T08:19:00.346265+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e16a5-db5a-6104-8000-80438cf20afc'}}, tasks=(PregelTask(id='4f1f6d11-7a59-fe0e-d080-1e36c3d70ca5', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'input': 'start'}, next=('step_1',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e16a5-db5a-6104-8000-80438cf20afc'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-12-25T08:19:00.345267+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'ch

In [10]:
#6.Rerun to show fault tolerant resume
final_state=graph.invoke(None ,{"configurable": {"thread_id": "thread-1"}})
print("final state: ", final_state)

⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
✅ Step 3 executed
final state:  {'input': 'start', 'step1': 'done', 'step2': 'done', 'step3': 'done'}


In [11]:
list(graph.get_state_history({"configurable": {"thread_id": "thread-1"}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done', 'step3': 'done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e16a9-1ab8-6499-8003-f8a927ff7bf9'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-12-25T08:20:27.520527+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e16a9-1ab3-630e-8002-f77e1833d71a'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e16a9-1ab3-630e-8002-f77e1833d71a'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-25T08:20:27.518439+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e16a5-db5c-67fe-8001-1b604ab24ab1'}}, tasks=(PregelTask(id='3c156bb4-d82e-eb38